In [4]:
import math
import random
import numpy as np

# Here I create a populations of neurons (size=80, 40 of which respond to stimulus 0 and 40 to stimulus 1).In the "homogenous"
#case the spikes are Poisson and the average number of spikes is uniform across the trial length. The only difference 
#between the response of neurons tuned to stim 0 vs. stim 1 is the Poisson rate parameter (80 vs. 100). In the "nonhomogenous" case
#what differentiates the two types of neurons is not their average firing rate (which is the same), but rather the difference
#

n_neurons = 80 #per condition
fr_stim = 60
dt = float(1)/1000
length_trial = 100 #in ms
num_trials = 400
#in the end we combine stim 1 and stim 2 so in total there will be 800 trials
#train on 600, test on 200

def is_odd(num):
    return bool(num % 2 != 0)

def sigmoid(a):
    return 1 / (1 + np.exp(-a))

def interleave(mat1,mat2,num_rows):
    unit_row = np.shape(mat1)[0]/num_rows;
    unit_col = np.shape(mat1)[1]
    interleaved = np.empty((0,unit_col))
    if (np.shape(mat1)[0]%num_rows)>0:
        print "Error"
        return
    else:
        start = 0
        for rows in range(unit_row):
            end = start + num_rows
            combined = np.append(mat1[start:end,:],mat2[start:end,:],axis=0)
            interleaved = np.concatenate((interleaved,combined),axis=0)
            start = start + num_rows
    return interleaved

def poissonGroup(fr,length_trial,n_neurons,dt,num_trials,type_):
    neurons_by_trials = n_neurons*(num_trials/2)
    if type_ == 1:
        truth_mat = np.random.uniform(0,1,(neurons_by_trials,length_trial)) < (fr*dt)
        fr_null = 0.9 * fr
        truth_mat2 = np.random.uniform(0,1,(neurons_by_trials,length_trial)) < (fr_null*dt)
    else:
        truth_mat2 = np.random.uniform(0,1,(neurons_by_trials,length_trial)) < (fr*dt)
        fr_null = 0.9 * fr
        truth_mat = np.random.uniform(0,1,(neurons_by_trials,length_trial)) < (fr_null*dt)
    poisson = interleave(truth_mat, truth_mat2,(n_neurons/2))
    return poisson

#homogenous case
#type_ refers to the subset of neurons that are most driven by the input
#1- first half, 2- second half
pop_stim1 = 1*poissonGroup(fr_stim,length_trial,n_neurons,dt,num_trials,1)
pop_stim2 = 1*poissonGroup(fr_stim,length_trial,n_neurons,dt,num_trials,2)

mean_stim1 = np.mean(np.sum(pop_stim1[:40],axis=1))
mean_stim2 = np.mean(np.sum(pop_stim1[40:80,:],axis=1))

#Non-homogenous case (rate changes every 10 ms, anti-correlated neurons)
step_time = length_trial/10
pop_corr1 = np.empty((n_neurons*num_trials,0)) #pre-allocate matrices
pop_corr2 = np.empty((n_neurons*num_trials,0))
for t in range(step_time):
    if is_odd(t):
        spikes1 = 1*poissonGroup(fr_stim,step_time,n_neurons,dt,num_trials,1)
        spikes2 = 1*poissonGroup(fr_stim,step_time,n_neurons,dt,num_trials,2)
    else:
        spikes1 = 1*poissonGroup(fr_stim,step_time,n_neurons,dt,num_trials,2)
        spikes2 = 1*poissonGroup(fr_stim,step_time,n_neurons,dt,num_trials,1) 
    pop_corr1 = np.concatenate((pop_corr1,spikes1),axis=1)
    pop_corr2 = np.concatenate((pop_corr2,spikes2),axis=1)
    

#Create a function to convert into 3-dimensional matrices of the form:
#[n_neurons,n_length,n_trials]
#n_neurons = n_neurons*2
def reshape(mat,n_neurons,length_trial,num_trials):
    reshaped = np.empty((n_neurons,length_trial,num_trials))
    for trial in range(num_trials):
        idx_end = (trial+1) * n_neurons
        idx_start = idx_end - n_neurons
        reshaped[:,:,trial] = mat[idx_start:idx_end,:]
    return reshaped


In [5]:
#For convenience we store all our data in 3D matrix format

#Model 1:
pop_stim1_3d = reshape(pop_stim1,n_neurons,length_trial,num_trials)
pop_stim2_3d = reshape(pop_stim2,n_neurons,length_trial,num_trials)

#Model2:
pop_corr1_3d = reshape(pop_corr1,n_neurons,length_trial,num_trials)
pop_corr2_3d = reshape(pop_corr2,n_neurons,length_trial,num_trials)

del pop_stim1, pop_stim2, pop_corr1, pop_corr2

In [ ]:
#This section visually validates that the method we used to generate the spikes worked properly

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

pref_stim1 = pop_stim1_3d[0:40,:,:]
null_stim1 = pop_stim1_3d[40:80,:,:]

pref_sum1 = np.sum(np.sum(pref_stim1,axis=2),axis=0)
null_sum1 = np.sum(np.sum(null_stim1,axis=2),axis=0)

pref_corr1 = pop_corr1_3d[0:40,:,:]
null_corr1 = pop_corr1_3d[40:80,:,:]
pref_sum1_corr = np.sum(np.sum(pref_corr1,axis=2),axis=0)
null_sum1_corr = np.sum(np.sum(null_corr1,axis=2),axis=0)

plt.subplot(2, 1, 1)
plt.title('2 models of encoding - Stimulus 1')
plt.plot((pref_sum1/200)*10)
plt.hold(True)
plt.plot((null_sum1/200)*10)
plt.ylim(30, 55)

plt.subplot(2,1,2)
plt.plot((pref_sum1_corr/200)*10)
plt.hold(True)
plt.plot((null_sum1_corr/200)*10)
plt.ylim(30, 55)

# These plots demonstrate two different encoding models. We hypothesize that logistic regression on rates will perform
#well on the first but not the second model
#The second model consists of a latent variable (preferred activity in each neuronal pool fluctuates and drives the 
# activity of the other population down). This is of course a cartoon model but serves to validate our method.
# blue is the preferred pool and green is the null. 
# For stimulus 2 these are just flipped

In [ ]:
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()  
clf = linear_model.LogisticRegression()
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1

def rate_feature_extraction(mat_3d,stim_type):
    all_data = np.empty((0,n_neurons+1))
    for trial in range(num_trials):
        trial_array = np.append(np.sum(mat_3d[:,:,trial],axis=1),stim_type)
        all_data = np.concatenate((all_data,[trial_array]),axis=0)
    return all_data

def test_accuracy(mat1,mat2,clf):
    all_ = np.concatenate((mat1,mat2),axis=0)
    X_all = all_[:,0:-1]
    y_all = all_[:,-1]
    #Shuffle and split
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=600)
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
    # apply same transformation to test data
    X_test = scaler.transform(X_test) 
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = float(np.sum(1*(y_pred==y_test)))/(np.shape(X_test)[0])
    return accuracy, clf, X_test, y_pred

#Model1:
#Independent rate model
#Apply a logistic regression to the spike counts
#Here each neuron's firing rate is a feature

#columns are spike counts, last column is stimulus type, rows are trials
rate_stim1 = rate_feature_extraction(pop_stim1_3d,0)
rate_stim2 = rate_feature_extraction(pop_stim2_3d,1)

accuracy1, clf1, X_test1, y_pred1 = test_accuracy(rate_stim1,rate_stim2,clf)
class1 = clf.coef_[:,0:20]
class2 = clf.coef_[:,20:40]

sigmoid_pred = 1*(sigmoid(X_test1.dot(clf1.coef_.T) + clf1.intercept_.T) > 0.5)
sigmoid_pred = np.ravel(sigmoid_pred)
similarity = np.sum(1*(y_pred1==sigmoid_pred))

In [ ]:
# Correlation code model
# Here the average firing rates are the same but the pattern of firing is different for the subsets of the population
# However the pattern is such that using spike counts over the 100ms window does not capture the difference

clf = linear_model.LogisticRegression()
#empty clf

rate_stim1 = rate_feature_extraction(pop_corr1_3d,0)
rate_stim2 = rate_feature_extraction(pop_corr2_3d,1)

accuracy2, clf2, X_test2, y_pred2  = test_accuracy(rate_stim1,rate_stim2,clf)

In [ ]:
def spike_feature_extraction(mat_3d,stim_type):
    all_data = np.empty((0,(n_neurons*length_trial)+1))
    for trial in range(num_trials):
        trial_array = np.append(mat_3d[:,:,trial].ravel(),stim_type)
        all_data = np.concatenate((all_data,[trial_array]),axis=0)
    return all_data

spike_stim1 = spike_feature_extraction(pop_stim1_3d,0)
spike_stim2 = spike_feature_extraction(pop_stim2_3d,1)
accuracy3, clf3, X_test3, y_pred3 = test_accuracy(spike_stim1,spike_stim2,clf)


spike_corr1 = spike_feature_extraction(pop_corr1_3d,0)
spike_corr2 = spike_feature_extraction(pop_corr2_3d,1)
accuracy4, clf4, X_test4, y_pred4 = test_accuracy(spike_corr1,spike_corr2,clf)

In [ ]:
plt.plot([accuracy1,accuracy2,accuracy3,accuracy4])

In [ ]:
#Model 2 with restricted boltzmann machine
from sklearn.neural_network import BernoulliRBM
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline


def test_accuracy_rbm(mat1,mat2):
    logistic = linear_model.LogisticRegression()
    rbm = BernoulliRBM(random_state=0, verbose=True, n_components = 200, n_iter = 30)
    clf_rbm = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
    all_ = np.concatenate((mat1,mat2),axis=0)
    X_all = all_[:,0:-1]
    y_all = all_[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,train_size=600,random_state=0)
    clf_rbm.fit(X_train,y_train)
    y_pred = clf_rbm.predict(X_test)
    accuracy = float(np.sum(1*(y_pred==y_test)))/(np.shape(X_test)[0])
    return accuracy

accuracy = test_accuracy_rbm(spike_corr1,spike_corr2)




# Models we will use